In [1]:
!pip install transformers

     |████████████████████████████████| 665kB 2.7MB/s 
     |████████████████████████████████| 3.8MB 13.3MB/s 
     |████████████████████████████████| 890kB 42.6MB/s 
     |████████████████████████████████| 1.1MB 41.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=80b1a1da2c834c3820f263b247a5073ef661d56e3131eda4040243aa43d76546
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4264  100  4264    0     0  48454      0 --:--:-- --:--:-- --:--:-- 48454
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-nightly ...
Uninstalling torch-1.5.0+cu101:
Done updating TPU runtime: <Response [200]>
  Successfully uninstalled torch-1.5.0+cu101
Uninstalling torchvision-0.6.0+cu101:
  Successfully uninstalled torchvision-0.6.0+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly-cp36-cp36m-linux_x86_64.whl...
- [1 files][ 90.2 MiB/ 90.2 MiB]                                                
Operation completed over 1 objects/90.2 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly-cp36-cp36m-linux_x86_64.whl...
- [1 files][121.3 MiB/121.3 MiB]                                                
Operation completed over 1 objects/121.3 MiB.         

In [0]:
!export XLA_USE_BF16=1

In [2]:
!nvidia-smi

Tue Jun  2 08:55:10 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir("drive/My Drive/kaggle-tweet-sentiment-extraction/pretrainingRoberta")

In [0]:
!pwd

/content/drive/My Drive/kaggle-tweet-sentiment-extraction/pretrainingRoberta


In [0]:
!ls

 checkpoints   install.sh		        roberta.base.tar.gz.1
 data-bin     'loading pretrain model.ipynb'    roberta-base-train.sh
 fairseq       roberta.base		        roberta-pretraining
 gpt2_bpe      roberta-base-path	        runs
 gpt2-bpe.sh   roberta-base-pytorch_model.bin   tweet
 gpt2dict.sh   roberta.base.tar.gz	        Untitled0.ipynb


In [0]:
!ls tweet

tweet.bpe  tweet.raw


In [0]:
import torch
import transformers

In [6]:
config = transformers.RobertaConfig("roberta-base")
tokenizer = transformers.RobertaTokenizerFast.from_pretrained("roberta-base")
model = transformers.RobertaForMaskedLM.from_pretrained("roberta-base")

In [7]:
model.num_parameters()

125288025

In [8]:
%%time

dataset = transformers.LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="tweet/tweet.raw",
    block_size=192
)

CPU times: user 2.72 s, sys: 160 ms, total: 2.88 s
Wall time: 2.82 s


In [9]:
len(dataset)

40001

In [0]:
data_collator = transformers.DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [0]:
training_args = transformers.TrainingArguments(
    output_dir="roberta-pretraining2",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_gpu_train_batch_size=32,
    save_steps=500,
    #save_total_limit=2,
    tpu_num_cores=8,
)

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True
)

In [14]:
%%time
trainer.train()

{"loss": 1.4219340126514435, "learning_rate": 4.800159872102318e-05, "epoch": 0.3996802557953637, "step": 500}


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


{"loss": 2.6894581854343413, "learning_rate": 4.600319744204636e-05, "epoch": 0.7993605115907274, "step": 1000}



{"loss": 2.5488004240989683, "learning_rate": 4.400479616306955e-05, "epoch": 1.1990407673860912, "step": 1500}
{"loss": 2.48534485244751, "learning_rate": 4.200639488409273e-05, "epoch": 1.5987210231814548, "step": 2000}
{"loss": 2.4794766154289247, "learning_rate": 4.000799360511591e-05, "epoch": 1.9984012789768184, "step": 2500}



{"loss": 2.4207620306015016, "learning_rate": 3.8009592326139094e-05, "epoch": 2.3980815347721824, "step": 3000}
{"loss": 2.4023238065242767, "learning_rate": 3.601119104716227e-05, "epoch": 2.797761790567546, "step": 3500}



...

{"loss": 2.1318526520729066, "learning_rate": 1.8025579536370903e-05, "epoch": 6.394884092725819, "step": 8000}
{"loss": 2.1045805039405825, "learning_rate": 1.6027178257394086e-05, "epoch": 6.794564348521183, "step": 8500}



{"loss": 2.075486779987812, "learning_rate": 1.4028776978417266e-05, "epoch": 7.194244604316546, "step": 9000}
{"loss": 2.0494732534885407, "learning_rate": 1.2030375699440447e-05, "epoch": 7.59392486011191, "step": 9500}
{"loss": 2.051946146965027, "learning_rate": 1.003197442046363e-05, "epoch": 7.9936051159072745, "step": 10000}



{"loss": 2.0354797501564024, "learning_rate": 8.033573141486811e-06, "epoch": 8.393285371702637, "step": 10500}
{"loss": 2.0195086891651153, "learning_rate": 6.035171862509993e-06, "epoch": 8.792965627498, "step": 11000}



{"loss": 2.0078809962272643, "learning_rate": 4.036770583533174e-06, "epoch": 9.192645883293366, "step": 11500}
{"loss": 1.985365638256073, "learning_rate": 2.0383693045563552e-06, "epoch": 9.59232613908873, "step": 12000}
{"loss": 1.9704402570724486, "learning_rate": 3.996802557953637e-08, "epoch": 9.992006394884093, "step": 12500}


CPU times: user 1h 28min 41s, sys: 49min 39s, total: 2h 18min 21s
Wall time: 2h 21min 27s


TrainOutput(global_step=12510, training_loss=2.195221625224387)

In [0]:
config = transformers.RobertaConfig("roberta-base")
tokenizer = transformers.RobertaTokenizerFast.from_pretrained("roberta-base")
model = transformers.RobertaForMaskedLM.from_pretrained("roberta-pretraining2/checkpoint-12500")

In [0]:
training_args = transformers.TrainingArguments(
    output_dir="roberta-pretraining3",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_gpu_train_batch_size=32,
    save_steps=500,
    #save_total_limit=2,
    tpu_num_cores=8,
)

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True
)

In [19]:
%%time
trainer.train()

{"loss": 0.9491718378141523, "learning_rate": 4.800159872102318e-05, "epoch": 0.3996802557953637, "step": 500}


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


{"loss": 1.5252184431552887, "learning_rate": 4.600319744204636e-05, "epoch": 0.7993605115907274, "step": 1000}



{"loss": 1.4741723943948746, "learning_rate": 4.400479616306955e-05, "epoch": 1.1990407673860912, "step": 1500}
{"loss": 1.4406549488306046, "learning_rate": 4.200639488409273e-05, "epoch": 1.5987210231814548, "step": 2000}
{"loss": 1.4420306466817856, "learning_rate": 4.000799360511591e-05, "epoch": 1.9984012789768184, "step": 2500}



{"loss": 1.410470760345459, "learning_rate": 3.8009592326139094e-05, "epoch": 2.3980815347721824, "step": 3000}
{"loss": 1.4271117861270906, "learning_rate": 3.601119104716227e-05, "epoch": 2.797761790567546, "step": 3500}



...

{"loss": 1.4819220886230469, "learning_rate": 1.8025579536370903e-05, "epoch": 6.394884092725819, "step": 8000}
{"loss": 1.5041748136281967, "learning_rate": 1.6027178257394086e-05, "epoch": 6.794564348521183, "step": 8500}



{"loss": 1.5226727382689715, "learning_rate": 1.4028776978417266e-05, "epoch": 7.194244604316546, "step": 9000}
{"loss": 1.5507877855300902, "learning_rate": 1.2030375699440447e-05, "epoch": 7.59392486011191, "step": 9500}
{"loss": 1.5964709942340851, "learning_rate": 1.003197442046363e-05, "epoch": 7.9936051159072745, "step": 10000}



{"loss": 1.6472276846170426, "learning_rate": 8.033573141486811e-06, "epoch": 8.393285371702637, "step": 10500}
{"loss": 1.6955300372838975, "learning_rate": 6.035171862509993e-06, "epoch": 8.792965627498, "step": 11000}



{"loss": 1.7488322575092317, "learning_rate": 4.036770583533174e-06, "epoch": 9.192645883293366, "step": 11500}
{"loss": 1.7922706338167191, "learning_rate": 2.0383693045563552e-06, "epoch": 9.59232613908873, "step": 12000}
{"loss": 1.851891249537468, "learning_rate": 3.996802557953637e-08, "epoch": 9.992006394884093, "step": 12500}


CPU times: user 1h 28min 19s, sys: 49min 16s, total: 2h 17min 36s
Wall time: 2h 20min 31s


TrainOutput(global_step=12510, training_loss=1.500131258247196)